# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 44 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.17173


extracting tarball to tmp_2505.17173... done.


G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']


Found 89 bibliographic references in tmp_2505.17173/paper.bbl.
Retrieving document from  https://arxiv.org/e-print/2505.17415


extracting tarball to tmp_2505.17415... done.
Retrieving document from  https://arxiv.org/e-print/2505.17428
extracting tarball to tmp_2505.17428... done.
Retrieving document from  https://arxiv.org/e-print/2505.17843


extracting tarball to tmp_2505.17843...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.17173-b31b1b.svg)](https://arxiv.org/abs/2505.17173) | **Probing the origins. II. Unravelling lithium depletion and stellar motion: Intrinsic stellar properties drive depletion, not kinematics**  |
|| M. L. L. Dantas, et al. -- incl., <mark>G. Guiglion</mark> |
|*Appeared on*| *2025-05-26*|
|*Comments*| *Accepted for publication in A&A. 12 pages, 6 figs, 3 tables. Abridged abstract to fit ArXiV's requirements*|
|**Abstract**|            In Paper I, we classified a stellar sample from the thin disc with a broad range in metallicity as being churned outward or inward, or blurred/undisturbed. In this paper (Paper II), we delve deeper by analysing our entire metallicity-stratified sample along with their dynamic properties, focusing on the connection between radial migration and Li depletion. We analyse the chemo-dynamics of a set of 1188 thin disc dwarf stars observed by the \textit{Gaia}-ESO survey, previously classified into six metallicity-stratified groups via Hierarchical Clustering (HC). We examine several features, such as effective temperatures, masses, and dynamic properties. We also implement a parametric survival analysis using penalised splines (logistic distribution) to quantify how stellar properties and motion (or migration) direction jointly influence Li depletion patterns. We find that stars in our sample that appear to have churned outward are predominantly Li-depleted, regardless of their metallicities. These stars are also the oldest, coldest, and least massive compared to those in the same HC group that have either churned inward or kept their orbital radii. Our survival analysis confirms temperature as the primary driver of Li depletion, followed by metallicity and age, while migration direction shows negligible influence. The increasing proportion of outward-churned stars with higher metallicity (and older ages) indicates their dominant influence on the overall trend observed in the [Fe/H]-A(Li) space for stellar groups with [Fe/H]>0. The survival model reinforces that the observed Li depletion stems primarily from intrinsic stellar properties (cool temperatures, higher metallicity, old ages) rather than migration history. This suggests the metallicity-dependent depletion pattern emerges through stellar evolution rather than Galactic dynamical processes.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.17415-b31b1b.svg)](https://arxiv.org/abs/2505.17415) | **Model-Independent Dark Energy Measurements from DESI DR2 and Planck 2015 Data**  |
|| <mark>Y. Wang</mark>, K. Freese |
|*Appeared on*| *2025-05-26*|
|*Comments*| *11pages, 4 figures. We welcome suggestions for the addition of relevant references*|
|**Abstract**|            Using DESI DR2 baryon acoustic oscillation distance measurements and Planck cosmic microwave background distance priors, we have measured the dark energy density $\rho_X(z)$ and dark energy equation of state $w_X(z)$ as free functions of redshift (smoothly interpolated from values at $z=0, 1/3, 2/3, 1, 4/3, 2.33$), and find both to be consistent with a cosmological constant, with only deviations of 1.2$\sigma$ for $\rho_X(z)$ and 1.9$\sigma$ for $w_X(z)$ at $z=2/3$. We confirm our earlier finding in Wang & Freese (2006) that $w_X(z)$ is significantly less constrained by data than $\rho_X(z)$. Our results differ noticeably from those of the DESI Collaboration, in which they used the same DESI DR2 data combined with Planck data and found a 3.1$\sigma$ deviation from a cosmological constant, a finding which is the consequence of their assuming the parametrization $w_X(z)=w_0+w_a(1-a)$. Our results indicate that assuming a linear $w_X(z)$ could be misleading and precludes discovering how dark energy actually varies with time at higher redshifts. In our quest to discover the physical nature of dark energy, the most urgent goal at present is to determine definitively whether dark energy density varies with time. We have demonstrated that it is of critical importance to measure dark energy density as a free function of redshift from data. Future galaxy redshift surveys by Euclid and Roman at higher redshifts will significantly advance our understanding of dark energy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.17428-b31b1b.svg)](https://arxiv.org/abs/2505.17428) | **Fast inflowing ionized absorber tracing the gas dynamics at sub-parsec scale around Mrk 3**  |
|| F. Shi, M. Guainazzi, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-05-26*|
|*Comments*| *11 pages, 4 figures, submitted to ApJ*|
|**Abstract**|            Accretion onto supermassive black hole (SMBH) releases energy via radiation, jets or winds, providing feedback effects on the circumnuclear gas environment. However, the dynamics of accreting gas on sub-parsec scales near the SMBH remains poorly understood. With high-resolution X-ray spectra of Mrk 3, we detect a fast inflowing ionized absorber characterized by redshifted Fe XXV and Fe XXVI absorption lines. Photoionization modeling reveals the inflowing absorber is located at $\lesssim0.04-0.74\rm~pc$ and is decelerating from $6.1\pm0.5\times10^3\rm~km~s^{-1}$ to $3.4\pm0.3\times10^3\rm~km~s^{-1}$ over 11 years. Only $\sim0.6$\%--$3$\% of the inflowing material is estimated to reach the event horizon. This direct evidence of sub-parsec scale fueling inflow bridges the gap between the torus and the outer accretion disk. Furthermore, a $0.86$-keV gas component with sub-solar metallicity ($Z\sim0.22$), outflowing at a velocity of $\sim330\rm~km~s^{-1}$, is detected in the soft X-ray band. It may corresponds to the [O III] biconical outflow in the narrow-line region. A putative ultra-fast disk wind outside our line-of-sight, or clouds within the broad-line region, are promising candidates for decelerating the inflowing absorber.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.17843-b31b1b.svg)](https://arxiv.org/abs/2505.17843) | **An improved Seyfert-LINER classification line in the [N~{\sc ii}] BPT diagram**  |
|| P. Cheng, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-05-26*|
|*Comments*| **|
|**Abstract**|            In this manuscript, an improved Seyfert-LINER classification line (= S-L line) is proposed in the [N~{\sc ii}] BPT diagram, based on a sample of 47,968 low redshift narrow emission line galaxies from SDSS DR16, motivated by different S-L lines reported in the [N~{\sc ii}] BPT diagram through different methods. The method proposed by Kewley et al. in 2006 is firstly applied, however, the method cannot lead to an accepted S-L line in the [N~{\sc ii}] BPT diagram. Meanwhile, the S-L lines proposed by Schawinski et al. in 2007 and Cid Fernandes et al. in 2010 in the [N~{\sc ii}] BPT diagram are different from each other. Therefore, it is meaningful to check which proposed S-L line is better or to determine an improved one in the [N~{\sc ii}] BPT diagram by a new method. In this manuscript, Seyferts and LINERs that have already been classified in the [S~{\sc ii}] and/or [O~{\sc i}] BPT diagrams can be visualized in the [N~{\sc ii}] BPT diagram, leading the intersection boundary of the two contour maps to be considered as the S-L line in the [N~{\sc ii}] BPT diagram. Rather than the previously proposed S-L lines, the new S-L line can lead to more efficient and harmonious classifications of Seyferts and LINERs, especially in the composite galaxy region, in the [N~{\sc ii}] BPT diagram. Furthermore, based on the discussed S-L lines, the number ratio of Type-2 Seyferts to Type-2 LINERs differs significantly from that of Type-1 Seyferts to Type-1 LINERs in the [N~{\sc ii}] BPT diagram, suggesting that about 90$\%$ of Type-2 LINERs are non-AGN-related objects, true Type-2 AGNs, or objects exhibiting both Seyfert and LINER characteristics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.17173/./figs/lithium_feh_comparison.png', 'tmp_2505.17173/./figs/combined_li_correlations.png', 'tmp_2505.17173/./figs/lithium_teff_movement.png']
copying  tmp_2505.17173/./figs/lithium_feh_comparison.png to _build/html/
copying  tmp_2505.17173/./figs/combined_li_correlations.png to _build/html/
copying  tmp_2505.17173/./figs/lithium_teff_movement.png to _build/html/
exported in  _build/html/2505.17173.md
    + _build/html/tmp_2505.17173/./figs/lithium_feh_comparison.png
    + _build/html/tmp_2505.17173/./figs/combined_li_correlations.png
    + _build/html/tmp_2505.17173/./figs/lithium_teff_movement.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\ali}{A(Li)}$
$\newcommand{\rgui}{\langle R_{\rm{g}} \rangle}$
$\newcommand{\rbirth}{\langle R_{\rm{b}} \rangle}$
$\newcommand{\eccentricity}{\langle e \rangle}$
$\newcommand{\zmax}{\langle Z_{\rm{max}} \rangle}$
$\newcommand{\lz}{\langle L_z \rangle}$
$\newcommand{\mass}{\langle M \rangle }$
$\newcommand{\age}{\overline{t}_{\star}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.}$</div>



<div id="title">

# Probing the origins: II. Unravelling lithium depletion and stellar motion: Intrinsic stellar properties drive depletion, not kinematics

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.17173-b31b1b.svg)](https://arxiv.org/abs/2505.17173)<mark>Appeared on: 2025-05-26</mark> -  _Accepted for publication in A&A. 12 pages, 6 figs, 3 tables. Abridged abstract to fit ArXiV's requirements_

</div>
<div id="authors">

M. L. L. Dantas, et al. -- incl., <mark>G. Guiglion</mark>

</div>
<div id="abstract">

**Abstract:** Lithium (Li) is a complex yet fragile element, with many production pathways but being easily destroyed in stars. Previous studies noted that the top envelope of the distribution of Li abundances [ A(Li) ] in super-solar metallicity dwarf stars shows signs of Li depletion, contrary to expectations. This depletion is thought to result from the interplay between stellar evolution and radial migration. In Paper I, we classified a stellar sample from the thin disc with a broad range in metallicity as being churned outward or inward, or blurred/undisturbed. In this paper (Paper II), we delve deeper by analysing our entire metallicity-stratified sample along with their dynamic properties, focusing on the connection between radial migration and Li depletion. We analyse the chemo-dynamics of a set of 1188 thin disc dwarf stars observed by the _Gaia_ -ESO survey, previously classified into six metallicity-stratified groups via Hierarchical Clustering (HC), ranging from metal-poor to super-metal-rich. We examine several features, such as effective temperatures, masses, and dynamic properties. We also implement a parametric survival analysis using penalised splines (logistic distribution) to quantify how stellar properties and motion (or migration) direction jointly influence Li depletion patterns. We find that stars in our sample that appear to have churned outward are predominantly Li-depleted, regardless of their metallicities. These stars are also the oldest, coldest, and least massive compared to those in the same HC group that have either churned inward or kept their orbital radii. Our survival analysis confirms temperature as the primary driver of Li depletion, followed by metallicity and age, while migration direction shows negligible influence. Additionally, the proportion of outward-churned stars increases with increasing metallicity, making up more than 90 \% of our sample in the most metal-rich group. The increasing proportion of outward-churned stars with higher metallicity (and older ages) indicates their dominant influence on the overall trend observed in the [ Fe/H ] -A(Li) space for stellar groups with [ Fe/H ] >0. The survival model reinforces that the observed Li depletion stems primarily from intrinsic stellar properties (cool temperatures, higher metallicity, old ages) rather than migration history. This suggests the metallicity-dependent depletion pattern emerges through stellar evolution rather than Galactic dynamical processes.

</div>

<div id="div_fig1">

<img src="tmp_2505.17173/./figs/lithium_feh_comparison.png" alt="Fig1" width="100%"/>

**Figure 1. -** Median lithium abundances ($\langle \rm{$\ali$} \rangle$) versus median metallicity ($\langle \rm{$\feh$} \rangle$) with their respective median errors for all the stellar groups in our sample, stratified by HC groups and Li detection (orange markers indicate detected values, purple markers indicate upper limits). Left panel: star-shaped markers depict $\langle \rm{$\ali$} \rangle$ vs $\langle \rm{$\feh$} \rangle$ for the entire sample, with each marker annotated to indicate the corresponding HC group. Right panel: Similar to the left panel, but further stratified by stellar movement. Circle markers represent stars that have moved inward, x-shaped markers indicate stars that have moved outward, and square markers depict stars with birth radius similar to their current Galactocentric distance (`Equal'). In cyan, we additionally display the abundances for the Solar photosphere ($\bigodot$), meteorites ($\bigotimes$), the Spite plateau (dashed horizontal line), and a newer SBBN estimate for $\ali$ from [Pitrou, et. al (2021)]() at 2.7 dex (dot-dashed horizontal line). Note that upper limits do not include error estimations for lithium since no de facto detection was made, as shown in all other plots of the paper. (*fig:li_feh_all*)

</div>
<div id="div_fig2">

<img src="tmp_2505.17173/./figs/combined_li_correlations.png" alt="Fig4" width="100%"/>

**Figure 4. -** Heatmaps displaying the correlations between several parameters for all the stars in the sample stratified by Li detection (detected and upper limits, respectively, from the left to right). We display the values of $\langle \rm{$\feh$} \rangle$, $\langle \rm{$\ali$} \rangle$, $\age$, $\langle M \rangle$, $\langle T_{\rm eff} \rangle$, $\eccentricity$, $\zmax$, $\lz$, and direction. We considered direction as a numerical value to classify stars churned outward (1) or inward (-1), or blurred/undisturbed (0). (*fig:li_correlations*)

</div>
<div id="div_fig3">

<img src="tmp_2505.17173/./figs/lithium_teff_movement.png" alt="Fig2" width="100%"/>

**Figure 2. -** Median lithium abundances ($\langle \rm{$\ali$} \rangle$) versus median effective temperatures ($\langle T_{\rm{eff}} \rangle$) for all the stellar groups in our sample, stratified by metallicity (through the HC) and Li detection (orange markers indicate detected values, purple markers indicate upper limits). Circle markers represent stars that have moved inward, X-shaped markers indicate stars that have moved outward, and square markers depict stars with similar birth and current Galactocentric distances (`Equal'). We additionally display the Sun within this parameter space in cyan ($\bigodot$), considering  $T_{\rm{eff, \odot}} = 5773 \pm 16$ K  ([Asplund, Amarsi and Grevesse 2021]()) ; the Spite plateau can be seen through the dashed horizontal cyan line, as well as the SBBN estimate by \citet[][dot-dashed line]{Pitrou2021}. (*fig:li_teff_all*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.17173"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

499  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
